# constrActive optimization

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import scipy as sp
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')

# Set up input graphs

In [2]:
from toolz import curry, pipe
from eden_chem.io.pubchem import download
from eden_chem.obabel import load as babel_load

download_active = curry(download)(active=True)
download_inactive = curry(download)(active=False)

def get_pos_graphs(assay_id): return pipe(assay_id, download_active, babel_load, list)
def get_neg_graphs(assay_id): return pipe(assay_id, download_inactive, babel_load, list)

In [3]:
from sklearn.neighbors import NearestNeighbors
from eden.graph import Vectorizer
import random
from constrActive import min_similarity_selection
from sklearn.metrics.pairwise import cosine_similarity

def outliers(graphs, k=3):
    vec = Vectorizer(r=3, d=3, normalization=False, inner_normalization=False, n_jobs=1)
    x = vec.transform(graphs)
    knn = NearestNeighbors()
    knn.fit(x)
    neigbhbors = knn.kneighbors(x, n_neighbors=k, return_distance=False)
    outlier_list = []
    non_outlier_list = []
    for i,ns in enumerate(neigbhbors):
        not_outlier = False
        for n in ns[1:]:
            if i in list(neigbhbors[n,:]):
                not_outlier=True
                break
        if not_outlier is False:
            outlier_list.append(i)
        else:
            non_outlier_list.append(i)
    return outlier_list, non_outlier_list

def select_non_outliers(graphs, k=3):
    outlier_list, non_outlier_list = outliers(graphs, k)
    graphs = [graphs[i] for i in non_outlier_list]
    logging.info('outlier removal:%d'%len(graphs))
    return graphs

def remove_similar_pairs(graphs):
    vec = Vectorizer(r=3, d=3, normalization=False, inner_normalization=False, n_jobs=1)
    x = vec.transform(graphs)
    matrix = cosine_similarity(x)
    scores = np.array([1]*len(graphs))
    ids = min_similarity_selection(matrix, scores=scores, max_num=len(graphs)/2)
    graphs = [graphs[i] for i in ids]
    logging.info('simila pairs removal:%d'%len(graphs))
    return graphs
    
def trim_by_size(graphs, fraction_to_remove=.1):
    frac = 1.0 - fraction_to_remove/2
    size = len(graphs)
    graphs = sorted(graphs, key=lambda g:len(g))[:int(size*frac)]
    graphs = sorted(graphs, key=lambda g:len(g), reverse=True)[:int(size*frac)]
    logging.info('trimming by size:%d'%len(graphs))
    return graphs

def trim_by_num(graphs, max_size):
    if len(graphs) > max_size:
        graphs = random.sample(graphs, max_size)
    logging.info('trimming by num:%d'%len(graphs))
    return graphs
    
def pre_process(graphs, fraction_to_remove=.1, n_neighbors_for_outliers=3, max_size=500):
    logging.info('original size:%d'% len(graphs))
    graphs = trim_by_size(graphs, fraction_to_remove)
    graphs = select_non_outliers(graphs, k=n_neighbors_for_outliers)
    graphs = remove_similar_pairs(graphs)
    return graphs

In [4]:
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors

def setup(graphs=None, k=None, m=None, vectorizer=None):
    x = vectorizer.transform(graphs)
    knn = NearestNeighbors()
    knn.fit(x)
    neigbhbors = knn.kneighbors(x, n_neighbors=k+m+1+1+1, return_distance=False)
    return neigbhbors

def select(id, neigbhbors, graphs, vectorizer, k=None, m=None):
    sel_neigbhbors = neigbhbors[id][1:]
    reference_graph = graphs[id]
    reference_vec = vectorizer.transform([reference_graph])[0]
    reference_graphs = [graphs[i] for i in sel_neigbhbors[:k]]
    reference_vecs = vectorizer.transform(reference_graphs)
    distances = euclidean_distances(reference_vec, reference_vecs)[0]
    pos_graphs = [graphs[i] for i in sel_neigbhbors[k:k+m/2]]
    neg_graphs = [graphs[i] for i in sel_neigbhbors[k+m/2:k+m]]
    seed_graph = graphs[sel_neigbhbors[k+m+1]]
    return reference_graph, seed_graph, reference_graphs, distances, pos_graphs, neg_graphs

In [5]:
from pareto_graph_optimizer import MultiObjectiveCostEstimator
from pareto_graph_optimizer import optimize_desired_distances
import time

def estimate_reconstruction_distance(id, neigbhbors, all_graphs, grammar, vectorizer, k=5, m=100):
    ts = time.time()
    res = select(id, neigbhbors, all_graphs, vectorizer, k=k, m=m)
    reference_graph, start_graph, reference_graphs, desired_distances, loc_pos_graphs, loc_neg_graphs = res

    cost_estimator = MultiObjectiveCostEstimator(vectorizer, improve=True).fit(loc_pos_graphs, loc_neg_graphs)

    res = optimize_desired_distances(start_graph, desired_distances, reference_graphs, vectorizer, grammar, cost_estimator)
    reference_graphs, pareto_set_graphs = res

    reference_vec = vectorizer.transform([reference_graph])[0]
    pareto_set_vecs = vectorizer.transform(pareto_set_graphs)
    distances = euclidean_distances(reference_vec, pareto_set_vecs)[0]
    min_dist = min(distances)/min(desired_distances)
    te = time.time()
    return (min_dist, id, te-ts)

In [6]:
from pareto_graph_optimizer import GrammarWrapper

def make_grammars(min_counts, vectorizer):
    grammars=dict()
    for min_count in min_counts:
        grammar = GrammarWrapper(vectorizer,radius_list=[0,1,2,3],thickness_list=[2],
                                 min_cip_count=min_count,min_interface_count=min_count)
        grammar.fit(all_graphs)
        grammars[min_count] = grammar
        logging.info('min count threshold: %d   grammar: %s' % (min_count, grammar))
    return grammars

In [7]:
import multiprocessing
from eden import apply_async

def reconstruction_error_experiment(dim, all_graphs, k=5, m=20, grammar=None, vectorizer=None):
    neigbhbors = setup(graphs=all_graphs, k=k, m=m, vectorizer=vectorizer)

    pool = multiprocessing.Pool()
    res = [apply_async(
        pool, estimate_reconstruction_distance, args=(id, neigbhbors, all_graphs, grammar, vectorizer, k, m))
           for id in range(dim)]
    min_dists=[]
    for p in res:
        min_dist,id,dt = p.get()
        min_dists.append(min_dist)
        logging.debug('id:%3d min distance:%.2f  (%.2f sec)'%(id, min_dist, dt))
    pool.close()
    pool.join()
    return min_dists

def display_results(min_dists, info_tuple):
    k, m, duration = info_tuple
    logging.info('Runtime: %.2f sec' % duration)
    logging.info('k:%d  m:%d' % (k,m))
    dim = len(min_dists)
    logging.info('Avg: %.2f'% np.mean(min_dists))
    zc = sum(1 for d in min_dists if d==0)
    logging.info('Perfect reconstruction in %d cases over %d (%.2f)'%(zc,dim, float(zc)/dim))
    bc = sum(1 for d in min_dists if d<1)
    logging.info('Better than 1-NN reconstruction in %d cases over %d (%.2f)'%(bc,dim, float(bc)/dim))


# Setup

In [8]:
assay_id = '651610' #apr93 23k mols
assay_id = '1834'   #apr90 500 mols
assay_id = '624466' #apr88 p5k n23k
assay_id = '588350' #apr86 p1k n600
assay_id = '449764' #apr85
assay_id = '492952' #apr85
assay_id = '463112' #apr82
assay_id = '463213' #apr70
assay_id = '119'    #apr60 30k mols
assay_id = '1224857'#apr10
assay_id = '2326'   #apr03 200k mols

#-------------------------------------------------
# choose
assay_id = '624466' #apr88 p5k n23k
assay_id = '588350' #apr86 p1k n600

In [9]:
# remove outliers, remove paiwise too similar, cluster in 2 and split in train and test
# induce grammar only on train
# reconstruct test instances 

In [10]:
%%time
# extract pos and neg graphs
pos_graphs, neg_graphs = get_pos_graphs(assay_id), get_neg_graphs(assay_id)

# remove too large and too small graphs and outliers
args=dict(fraction_to_remove=.1, n_neighbors_for_outliers=3, max_size=300)
logging.info('Positive graphs')
pos_graphs = pre_process(pos_graphs, **args)
logging.info('Negative graphs')
neg_graphs = pre_process(neg_graphs, **args)
all_graphs = pos_graphs+neg_graphs

# setup colors
from eden.display import map_labels_to_colors
label_colors = map_labels_to_colors(pos_graphs + neg_graphs)
from graphlearn.utils import draw
draw_params = dict(n_graphs_per_line=7, size=9, colormap='Paired', vertex_color='_labels_', vertex_color_dict=label_colors, vertex_alpha=0.5, edge_alpha=0.2)

Positive graphs
original size:1232
trimming by size:1170
outlier removal:649
simila pairs removal:324
Negative graphs
original size:634
trimming by size:602


KeyboardInterrupt: 

In [ ]:
%%time
from eden.graph import Vectorizer
vectorizer = Vectorizer(r=3, d=3,normalization=False,inner_normalization=False,n_jobs=1)

min_counts=range(3,0,-1)
grammars = make_grammars(min_counts, vectorizer)

# Empirical Analysis

In [ ]:
import time

num_test_instances=50
k=5

for min_count in sorted(grammars, reverse=True):
    for m in [20,80,160]:
        logging.info('-'*130)
        grammar = grammars[min_count]
        logging.info('grammar: min_count: %d    %s' % (min_count, grammar))
        ts = time.time()
        min_dists = reconstruction_error_experiment(num_test_instances, all_graphs, k, m, grammar, vectorizer)
        te = time.time()
        duration = te-ts
        info_tuple = (k, m, duration)
        display_results(min_dists, info_tuple)

---

# Results

### Grammar
 #instances: 432   #interfaces: 10660   #cores: 5588   #core-interface-pairs: 16311

CPU times: user 13.6 s, sys: 972 ms, total: 14.6 s

Wall time: 19.2 s


### Results
avg: 0.13

Perfect reconstruction in 87 cases over 100 (0.87)

Better than 1-NN reconstruction in 91 cases over 100 (0.91)

CPU times: user 2h 14min 56s, sys: 6min 42s, total: 2h 21min 38s

Wall time: 2h 32min 41s